In [1]:
from collections import defaultdict, namedtuple
from importlib import reload
from pathlib import Path
import sys

from sklearn.datasets import fetch_20newsgroups

sys.path.insert(0, str(Path.cwd().parents[0] / 'src')) 
import crypto as cg; reload(cg)

<module 'crypto' from '/Users/chris/Documents/cryptogram-solver/src/crypto.py'>

In [81]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=20180827)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=20180827)

In [82]:
reload(cg)
tokenizer = cg.Tokenizer(char_ngram_range=(1, 6), word_ngram_range=(1, 1), vocab_size=1000000)
tokenizer.fit(news_train['data'][:250])
print(len(tokenizer.vocab))
print(tokenizer.totals)

100%|██████████| 250/250 [00:04<00:00, 54.88it/s]


85485
defaultdict(<class 'int'>, {(<NgramKind.word: 0>, 1): 82631, (<NgramKind.char: 1>, 1): 467854, (<NgramKind.char: 1>, 2): 396195, (<NgramKind.char: 1>, 3): 329212, (<NgramKind.char: 1>, 4): 268452, (<NgramKind.char: 1>, 5): 208407, (<NgramKind.char: 1>, 6): 153166})


In [8]:
doc2 = cg.Doc(news_test['data'][10])
# doc2 = prep.Doc(doc2.swap('a', 'b'))
# doc2.scramble(15)
doc2.text

'from: ez027993@dale.ucdavis.edu (gary built like villanueva huckabay)\nsubject: ron gant, stalling, and hirschbeck.\norganization: julio machado candlelight vigil society\nlines: 88\n\n(dale "seer" stephenson) writes:\n>while i can see why they want to cut down on the time spent walking around,\n>gant wasn\'t ordered back in the box "sooner", he was ordered back immediately.\n>as soon as he stepped out, hirschbeck told him to get back in the box.\n>now, gant doesn\'t take a lot of walks between pitches.  the only reason\n>he did then because he was *very* bothered by the call.  i expect his\n>concentration wasn\'t there yet, and in a crucial situation i imagine it\'s\n>best to be as calm as possible.  contributing factors would be gant\'s\n>bad day at the plate, bad year at the plate, and the braves long scoring\n>drought.\n\nand it is not hirschbeck\'s job to help gant with any of these difficulties.\nif gant can\'t gather his concentration for whatever reason, that just makes\nhim a

In [26]:
import re

In [42]:
'4536271809'.index('0')
'4536271809'.index('5')

8

In [70]:
del letters

In [ ]:
for idx in idx1:
    

In [20]:
def solve_cryptogram(text, tokenizer, max_epochs=50, is_debug=True):
    doc = cg.Doc(text)
    best_nll = tokenizer.calc_nll(doc.text)
    best_swap = ('a', 'a')
    epoch = 0
    improving = True
    while improving and epoch < max_epochs:
        improving = False
        print('*', end='')
        seen = set()
        for letter1 in doc.letters:
            for letter2 in LETTERS - set(letter1):
                swap = tuple(sorted([letter1, letter2]))
                if swap in seen:
                    continue
                seen.add(swap)
                new_text = doc.swap(*swap)
                nll = tokenizer.calc_nll(new_text)
                if nll < best_nll:
                    best_swap = swap
                    best_nll = nll
                    improving = True
        doc = cg.Doc(doc.swap(*best_swap))
        epoch += 1
        print(f'{doc.text}')
        print(f'best of epoch: {best_nll:0.5g}')
    print(f'\nfinal best ({epoch} epochs): {best_nll:0.5g}')
    return doc.text

In [21]:
text = doc2.text
text = 'Rbo rpktigo vcrb bwucja wj kloj hcjd, km sktpqo, cq rbwr loklgo vcgg cjqcqr kj skhcja wgkja wjd rpycja rk ltr rbcjaq cj cr. -- Roppy Lpwrsborr'
text = 'This is the story of a girl. Who cried a river and drowned the whole world. And while she looked so sad in photographs, I absolutely loved her when she smiled.'
doc = cg.Doc(text)
doc.scramble(25)
doc.text

'toyb yb tol btvan ve k wyap. iov caylj k aymla kgj javiglj tol iovpl ivapj. kgj ioypl bol pvvxlj bv bkj yg fovtvwakfob, y kdbvputlpn pvmlj ola iolg bol bsyplj.'

In [22]:
# text, options = solve_cryptogram(doc.text, tokenizer)
text = solve_cryptogram(doc.text, tokenizer)
print(text)

*toyb yb tol btran re k wyap. ior caylj k aymla kgj jariglj tol iorpl irapj. kgj ioypl bol prrxlj br bkj yg fortrwakfob, y kdbrputlpn prmlj ola iolg bol bsyplj.
best of epoch: 23.687
*toyb yb tol btran rj k wyap. ior cayle k aymla kge earigle tol iorpl irape. kge ioypl bol prrxle br bke yg fortrwakfob, y kdbrputlpn prmle ola iolg bol bsyple.
best of epoch: 23.076
*tomb mb tol btran rj k wmap. ior camle k amyla kge earigle tol iorpl irape. kge iompl bol prrxle br bke mg fortrwakfob, m kdbrputlpn pryle ola iolg bol bsmple.
best of epoch: 22.628
*toms ms tol stran rj k wmap. ior camle k amyla kge earigle tol iorpl irape. kge iompl sol prrxle sr ske mg fortrwakfos, m kdsrputlpn pryle ola iolg sol sbmple.
best of epoch: 22.308
*toms ms tol stran rj k wmap. gor camle k amyla kie eargile tol gorpl grape. kie gompl sol prrxle sr ske mi fortrwakfos, m kdsrputlpn pryle ola goli sol sbmple.
best of epoch: 22.058
*toms ms tol stran rj k wmap. cor gamle k amyla kie earcile tol corpl crape. kie comp

In [517]:
from collections import Counter

In [518]:
Counter(['a', 'b', 'c', 'c'])

Counter({'a': 1, 'b': 1, 'c': 2})

In [23]:
def softmax(options, temp=1):
    nums = [exp(x / temp) for x in options.values()]
    denom = sum(nums)
    choice_probs = [x / denom for x in nums]
    choice = random.choices(list(options.keys()), weights=choice_probs)[0]
    return choice


# options = {'a': 1, 'b': 2}
softmax(options, temp=0.05)

NameError: name 'options' is not defined

In [1775]:
softmax(options)

[4.5397868702434395e-05, 0.9999546021312976]


'b'